# Exercise 10: Multilingual Representation Learning

In this exercise, you will induce and explore multilingual text representations, cross-lingual word embeddings and multilingual encoders.

You should complete the parts of the exercise that are marked as **TODO**.
A correctly completed **TODO** gives 2 bonus points. Partially correct answers give 1 bonus point.
Some **TODO**s are inside a comment in a code block: Here, you should complete the line of code.
Other **TODO**s are inside a text block: Here, you should write a few sentences to answer the question.

**Important:** Some students were under the impression that you have to complete a TODO in a _single_ line of code. That is not the case, you can use as many lines as you need.

**Submission deadline:** 31.01.2022, 23:59 Central European Time

**Instructions for submission:** After completing the exercise, save a copy of the notebook as exercise10_multilinguality_MATRIKELNUMMER.ipynb, where MATRIKELNUMMER is your student ID number. Then upload the notebook to moodle (submission exercise sheet 10).

In order to understand the code, it can be helpful to experiment a bit during development, e.g., to print vectors, matrices, and tensors or their shapes. But please remove these changes before submitting the notebook. If we cannot run your notebook, or if a print statement is congesting stdout too much, then we cannot grade it. 

To make the most of this exercise, you should try to read and understand the entire code, not just the parts that contain a **TODO**. If you have questions, write them down for the exercise, which will happen in the week after the submission deadline.

**CUDA:** You can use a GPU for this exercise (on colab: Runtime -> Change Runtime Type -> GPU). This is not mandatory (nor particularly crucial for this exercise), but it may speed up the execution of the code. 

# Required libraries

When working with or any fast-changing software library, you should be extra careful to fix the library versions when you begin your project, and not change versions while you're developing.


## Data

This exercise requires various data files (monolingual embeddings, translation dictionaries, training and evaluation data for cross-lingual transfer for sentiment classification) which have been zipped and need to be obtained from: 

https://tinyurl.com/2p9x4crt

You need to place the content of the archive into a directory named **data** and place it in the same directory with this notebook (so that the files can be accessed by the code via a relative path "data/file_name"). 

In [3]:
!pip install sentence-transformers==2.1.0
!pip install numpy==1.20.0 # other numpy versions most likely also ok
!pip install pandas==1.2.2
!pip install scikit-learn==1.0

### Multilingual word embeddings

We will start by exploring multilingual word embedding spaces. We will start from monolingually-trained (that is, mutually unaligned) word vectors of several languages: English, German, Italian, and Croatian. 

Each monolingual embedding space is serialized in two files: a *pickled Python dictionary* (.vocab) that maps words to indices in the embedding matrix, and an embedding matrix (*a serialized 2D numpy array*, .vectors) that contains the actual embeddings. All embedding files should be in the **data** subdirectory, which should be in the same directory as this notebook. 

In [4]:
# we will load pre-trained serialized monolingual vectors
import pickle
import numpy, io, json
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data
# Load (that is, "unpickle") the vocabularies for all languages 
vocab_en = pickle.load(open("data/ft.wiki.en.300.vocab","rb"))#TODO: load the English vocabulary 
vectors_en = numpy.load(open("data/ft.wiki.en.300.vectors","rb"))#TODO: load the English embedding vectors (2D numpy array) 

# let's see how many words we have in the vocabulary
print(len(vocab_en))

# let's see the dimensions of the embedding matrix
print(vectors_en.shape)

199984
(199984, 300)


In [5]:
# let's see what the vector of some word looks like
word = "dog"
print("Index of " + word + ": " + str(vocab_en[word]))

vector = vectors_en[vocab_en[word.lower()]]#TODO: print the embedding vector of the above specified word 
#(currently "dog", but feel free to change to any other word)

print("Vector of " + word + ": ")
print(vector)


Index of dog: 2344
Vector of dog: 
[ 1.9083e-01  6.5282e-02 -2.6465e-01  1.1757e-01 -2.8956e-01  9.8984e-02
 -3.2181e-01 -2.7679e-01  7.2732e-02  2.1486e-01 -6.5308e-02  1.7848e-01
  1.2327e-01 -3.1839e-01  2.1547e-01 -1.8138e-01 -1.3029e-02 -7.6652e-02
  4.3791e-03 -5.3426e-03  2.8255e-01 -4.0270e-02 -3.2650e-01 -1.8132e-01
  2.3960e-01 -1.5806e-03 -1.1625e-01  1.7114e-01  2.9568e-01  2.1059e-01
  3.7243e-01  7.5459e-02 -7.6080e-02 -1.7011e-01  2.2723e-01 -3.4284e-02
 -1.2251e-01 -2.4276e-01 -3.3694e-03  3.5631e-01 -1.4193e-02 -1.7566e-01
  2.5879e-02  2.3394e-01  1.4237e-01  3.1281e-01  1.2284e-01 -2.6890e-01
  1.5700e-01  1.1626e-01  8.6337e-02  1.9307e-01 -2.5144e-01  9.4174e-02
 -5.6797e-03  4.1143e-01  4.1341e-01 -3.2283e-01  1.0963e-01  3.5649e-01
  2.8923e-02 -3.6037e-01  1.6095e-01  9.2830e-02 -2.0297e-02 -2.6669e-01
  2.5275e-01  1.9916e-01 -2.8537e-02  1.1633e-01  2.5918e-01 -2.8496e-01
  1.1494e-01 -3.7263e-01 -2.2344e-01  1.5158e-01  2.0357e-01  3.7800e-01
 -2.8566e-01 -9.

In [6]:
# let us now load vectors and vocabularies of a few other languages

# German
vocab_de = pickle.load(open("data/ft.wiki.de.300.vocab","rb"))#TODO: load the German vocabulary 
vectors_de = numpy.load(open("data/ft.wiki.de.300.vectors","rb"))#TODO: load the German embeddings (2D numpy array) 

# Italian
vocab_it = pickle.load(open("data/ft.wiki.it.300.vocab","rb"))#TODO: load the Italian vocabulary 
vectors_it = numpy.load(open("data/ft.wiki.it.300.vectors","rb"))#TODO: load the Italian embeddings (2D numpy array) 

# Croatian
vocab_hr = pickle.load(open("data/ft.wiki.hr.300.vocab","rb"))#TODO: load the Croatian vocabulary  
vectors_hr = numpy.load(open("data/ft.wiki.hr.300.vectors","rb"))#TODO: load the Croatian embeddings (2D numpy array) 

# let's see how many entries we have in vocabularies of languages
print("DE", len(vocab_de), vectors_de.shape)
print("IT", len(vocab_it), vectors_it.shape)
print("HR", len(vocab_hr), vectors_hr.shape)

# TODO: What is the dimensionality of the embeddings (same for all languages :)?
# ANS: 300

DE 199968 (199968, 300)
IT 199994 (199994, 300)
HR 199931 (199931, 300)


Are the vectors from individual monolingual embedding spaces **comparable**? **No, they are not**. Let's verify that. Let's compare vector similarities within language and across languages

In [7]:
# cosine similarity is a common measure of similarity in vector space in NLP
# we just define a function that compute the cosine of the angle between the two vectors
# cosine similarity is a dot-product between the vectors normalized by the Euclidean (L2) norm of each vector
def cosine_sim(vec1, vec2):
    norm1 = numpy.linalg.norm(vec1)
    norm2 = numpy.linalg.norm(vec2)
    return numpy.dot(vec1, vec2) / (norm1 * norm2)

In [8]:
# let's see some monolingual similarities

language = "de" # play with different languages, "en", "it", "hr"

# just a shortcut, so we don't have to change the variables with vectors/vocabularies, we merely change the "language" variable
vectors = vectors_en if language == "en" else vectors_de if language == "de" else vectors_it if language == "it" else vectors_hr
vocab = vocab_en if language == "en" else vocab_de if language == "de" else vocab_it if language == "it" else vocab_hr

word1 = "hund" # play with different words
word2 = "katze" # play with different words

vector1 = vectors[vocab[word1.lower()]]#TODO: get the vector of the first word (word1) 
vector2 = vectors[vocab[word2.lower()]]#TODO: get the vector of the second word (word2)  

sim = cosine_sim(vector1, vector2)#TODO: compute the cosine similarity between the embedding vectors of the two words 
print("Similarity between " + word1 + " and" + word2 + ": " + str(sim))


Similarity between hund andkatze: 0.7434614


In [9]:
# let's put all embeddings and vocabularies into one dictionary
# just for easy access
emb_dict = {"en" : (vocab_en, vectors_en), 
            "de" : (vocab_de, vectors_de), 
            "it" : (vocab_it, vectors_it), 
            "hr" : (vocab_hr, vectors_hr)}

# lets create a more general function for comparing similarities between words from any two langs
def word_similarity(lang1, word1, lang2, word2):
    vocab_1 = emb_dict[lang1][0]#TODO: get the vocabulary of the lang1
    vectors_1 = emb_dict[lang1][1]#TODO: get the embeddings of the lang1 
    
    vocab_2 = emb_dict[lang2][0]#TODO: get the vocabulary of the lang2
    vectors_2 = emb_dict[lang2][1]#TODO: get the embeddings of the lang2
    
    vector_word_1 = vectors_1[vocab_1[word1.lower()]]
    vector_word_2 = vectors_2[vocab_2[word2.lower()]]
    
    return cosine_sim(vector_word_1, vector_word_2)#TODO: return cosine similarity of vector_word_1 and vector_word_2
    


In [10]:
sim = word_similarity("de", "katze", "en", "cat") #TODO: compute the similarity between the German word "katze" and English word "cat"
print(sim)

0.039084166


So the independently built monolingual word embedding spaces of different languages are **not semantically aligned**. We need to **align them**. 

- We will do this by computing a **projection matrix** that rotates and translates one embedding space with respect to the other! 

- How do we know what we need to align? We provide some number of word translation pairs! 

In [11]:
import codecs

prefix_trans = "/work/gglavas/data/word_embs/yacle/translations/freq_split/pairwise/"

word_pairs_de_en =  [(l.strip().split("\t")[0], l.strip().split("\t")[1]) for l in codecs.open("data/translations.5k.de-en.tsv", "r", encoding = 'utf8', errors = 'replace').readlines()]
word_pairs_it_en =  [(l.strip().split("\t")[1], l.strip().split("\t")[0]) for l in codecs.open("data/translations.5k.en-it.tsv", "r", encoding = 'utf8', errors = 'replace').readlines()]
word_pairs_hr_en =  [(l.strip().split("\t")[1], l.strip().split("\t")[0]) for l in codecs.open("data/translations.5k.en-hr.tsv", "r", encoding = 'utf8', errors = 'replace').readlines()]


In [12]:
print(word_pairs_de_en)

[('von', 'of'), ('zu', 'to'), ('im', 'in'), ('zum', 'for'), ('ist', 'is'), ('auf', 'on'), ('Das', 'that'), ('durch', 'by'), ('Dies', 'this'), ('mit', 'with'), ('ich', 'i'), ('Sie', 'you'), ('es', 'it'), ('oder', 'or'), ('Sein', 'be'), ('sind', 'are'), ('beim', 'at'), ('alle', 'all'), ('haben', 'have'), ('Neu', 'new'), ('wir', 'we'), ('werden', 'will'), ('Zuhause', 'home'), ('uns', 'us'), ('Über', 'about'), ('ob', 'if'), ('Seite', 'page'), ('meine', 'my'), ('hat', 'has'), ('Suche', 'search'), ('frei', 'free'), ('aber', 'but'), ('unser', 'our'), ('ein', 'one'), ('andere', 'other'), ('machen', 'do'), ('Information', 'information'), ('Zeit', 'time'), ('er', 'he'), ('welche', 'which'), ('ihr', 'their'), ('Nachrichten', 'news'), ('benutzen', 'use'), ('Dort', 'there'), ('sehen', 'see'), ('damit', 'so'), ('Wer', 'who'), ('Netz', 'web'), ('ebenfalls', 'also'), ('bekommen', 'get'), ('pm', 'pm'), ('Aussicht', 'view'), ('c', 'c'), ('e', 'e'), ('zuerst', 'first'), ('gewesen', 'been'), ('würde', 'wo

In [13]:
# let's now create the matrices of aligned vectors of word translations, 
# given monolingual embeddings and word translation pairs

def align_word_vectors(src_vecs, src_vocab, trg_vecs, trg_vocab, trans_pairs):
    src_matrix =  []
    trg_matrix =  []
    
    # for each pair of words in our translation pairs
    for src_word, trg_word in trans_pairs:
        # add the vector of the source language word to the source matrix
        src_matrix.append(src_vecs[src_vocab[src_word.lower()]])
        # add the vector of the corresponding (translation) target language word to the target matrix
        trg_matrix.append(trg_vecs[trg_vocab[trg_word.lower()]])
        
    # return the row-aligned matrices (at the same index are vectors of mutual translations)
    # from these matrices, we will compute the projection/alignment matrix using the Procrustes method
    return numpy.array(src_matrix), numpy.array(trg_matrix)



In [14]:
src_de_en, trg_de_en = align_word_vectors(emb_dict["de"][1],emb_dict["de"][0],emb_dict["en"][1],emb_dict["en"][0],word_pairs_de_en)#TODO: call the align_word_vectors to align embeddings for given translation pairs 
                       # between German (source) and English (target)  

print(src_de_en.shape)
print(trg_de_en.shape)


(5000, 300)
(5000, 300)


In [15]:
# let's learn a projection matrix, given the aligned matrices of vectors of word translations
# we will use the so-called Procrustes solution of the alignment problem (i.e., finding the optimal projection matrix)
def get_projection_procrustes(src_mat, trg_mat):
    product = numpy.dot(numpy.transpose(src_mat),trg_mat)#TODO: complete this line so the result is a correct projection matrix from source to target embedding space
    U, S, V = numpy.linalg.svd(product)
    
    return numpy.matmul(U, V)


In [16]:
proj_mat_de_en = get_projection_procrustes(src_de_en, trg_de_en)#TODO: obtain the projection matrix between German and English using the 
                 # translation-aligned embedding matrices 

#TODO: what is the shape of the projection matrix? Write the code that shows it
print(proj_mat_de_en.shape)

(300, 300)


In [17]:
# let's now project the vectors of all the Italian words to the English vector space
proj_vectors_de = numpy.matmul(vectors_de, proj_mat_de_en)

# let's replace the original German vectors with the projected ones in the embeddings dictionary of all languages
emb_dict["de"] = (vocab_de, proj_vectors_de)

In [18]:
# let's see now what the German-English similarities look like after projection of German embeddings to the English emb. space
word_similarity("de", "katze", "en", "cat")

0.6753164

In [19]:
# let's perform the mapping to the English embedding space for the other two languages as well

## for Italian

# aligning vectors of word translations
src_it_en, trg_it_en = align_word_vectors(emb_dict["it"][1],emb_dict["it"][0],emb_dict["en"][1],emb_dict["en"][0],word_pairs_it_en)#TODO: align embeddings for translation pairs, Italian-English 
proj_mat_it_en = get_projection_procrustes(src_it_en, trg_it_en)#TODO: Compute the projection matrix from Italian to English embedding space
proj_vectors_it = numpy.matmul(vectors_de, proj_mat_de_en)#TODO: project the vectors of all Italian words to the English space, using the obtained projection matrix
emb_dict["it"] = (vocab_it, proj_vectors_it)#TODO: replace the original (unaligned) Italian vectors with the projected (aligned) ones

In [20]:
word_similarity("it", "gatto", "en", "cat")

0.17871049

In [21]:
## for Croatian

src_hr_en, trg_hr_en = align_word_vectors(emb_dict["hr"][1],emb_dict["hr"][0],emb_dict["en"][1],emb_dict["en"][0],word_pairs_hr_en)#TODO: align embeddings for translation pairs, Croatian-English  
proj_mat_hr_en = get_projection_procrustes(src_hr_en, trg_hr_en)#TODO: Compute the projection matrix from Croatian to English embedding space
proj_vectors_hr = numpy.matmul(vectors_hr, proj_mat_hr_en)#TODO: project the vectors of all Croatian words to the English space, using the obtained projection matrix
emb_dict["hr"] = (vocab_hr, proj_vectors_hr)#TODO: replace the original (unaligned) Croatian vectors with the projected (aligned) ones

In [22]:
print(word_similarity("hr", "mačka", "en", "cat"))

# note that words of all four languages are now embedded in the same (originally English) embedding space
# so we can semantically compare words between any two of these languages (not just X-EN)
print(word_similarity("hr", "pas", "de", "hund"))
print(word_similarity("it", "gatto", "hr", "mačka"))
print(word_similarity("de", "flughafen", "it", "aeroporto"))

0.5466043
0.5367054
0.141094
0.21152036


### Pretrained multilingual encoders!

Pretrained multilingual transformers enable the comparison of meaning of longer units of text in different languages. To this end, we will use pretrained transformers specialized for encoding sentence-level semantics: SentenceTransformers (package sentence-transformers). More information: 

https://www.sbert.net/

https://arxiv.org/pdf/1908.10084.pdf

https://arxiv.org/pdf/2004.09813.pdf


In [23]:
!export CUDA_VISIBLE_DEVICES="0"
from sentence_transformers import SentenceTransformer

# loading the pretrained sentence encoder (concretely, the DistilmUSE model, distilled from multilingual USE)
sent_encoder = SentenceTransformer('distiluse-base-multilingual-cased-v2')

2022-01-22 06:20:58.311688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-22 06:20:58.311708: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [24]:
sent_en = "Hello World"
sent_de = "Hallo Welt"
sent_es = "Hola mundo"

sentence_embeddings = sent_encoder.encode([sent_en,sent_de,sent_es])#TODO: encode (that is, obtain the embeddings for) the above three sentences using the loaded sent_encoder
print(sentence_embeddings.shape)

(3, 512)


In [25]:
import numpy

sim_en_de = cosine_sim(sentence_embeddings[0],sentence_embeddings[1])#TODO: Compute the cosine similarity between the embedding of the English sentence and the German sentence

sim_de_es = cosine_sim(sentence_embeddings[2],sentence_embeddings[1])#TODO: Compute the cosine similarity between the embedding of the Spanish sentence and the German sentence

print(sim_en_de)
print(sim_de_es)

0.97415096
0.9840379


### Cross-lingual transfer for downstream NLP tasks

We will now see how to exploit a multilingual representation space (i.e., our multilingual sentence encoder) to train a model for a text classification task on annotated data in one language and then use that classification model to make predictions for texts from other languages.

**Task**: sentiment classification of Amazon reviews

**Annotated training data**: in English

In [26]:
# let's load the data using the *pandas* library
# we have two files: the training dataset and testing dataset in English

# importing the Python's Pandas library for data loading and manipulation
import pandas as pd

# Step #1: loading our annotated reviews
train_data = pd.read_csv("data/labeled_train.txt", delimiter = '\t') # in our file, the values are actually TAB-separated
eval_data = pd.read_csv("data/labeled_test.txt", delimiter = '\t')

# let's see what our data actually looks like
train_data




,label,score,content
0,NEG,2.0,cons tips extremely easy on carpet and if you...
1,NEG,1.0,"It's a nice look, but it tips over very easil..."
2,NEG,1.0,I have bought and returned three of these uni...
3,NEG,1.0,"I knew these were inexpensive CD cases, but I..."
4,NEG,2.0,"I used a 25 pack of these doing DVD backups, ..."
...,...,...,...
1795,POS,5.0,I just recieved my HDMI cable and am very imp...
1796,POS,5.0,This is the perfect keyboard ( I know cuz I a...
1797,POS,5.0,SanDisk has done it again. They never seem to...
1798,POS,5.0,"Fast shipping, Very happy with the GARMIN. Th..."


In [27]:
# task: predict the binary sentiment label (POS or NEG) from the encoding of the review text produced by the Sentence Transformer
# as the classifier, we will use simple logistic regression model

# filtering just the text from the pandas dataframe
train_texts = list(train_data["content"])

# embedding all texts with the sentence encoder
train_embeddings = sent_encoder.encode(train_texts)#TODO: get the embeddings of the reviews (train_texts) with the sent_encoder

# converting labels from "POS" and "NEG" into numeric labels (0 and 1) for classification
train_labels = train_data["label"].tolist()
train_labels = [(1 if tl == "POS" else 0) for tl in train_labels]


In [28]:
# Now that we have input (embeddings of train texts) and labels, we can use them to train a classifier
# To this end, we will use a simple logistic regression classifier from scikit-learn

# We import the LogisticRegression class from scikit-learn
from sklearn.linear_model import LogisticRegression

# we now train ("fit") the logistic regression classifier by providing the training input (SBERT embeddings of our training set texts) and 
# corresponding sentiment labels (0 or 1)

classifier = LogisticRegression(C = 32, solver = 'lbfgs', max_iter = 1000)
classifier.fit(train_embeddings, train_labels)

# the result is a trained classifier, which we can examine more closely in the next steps and make predictions with
print(classifier)



LogisticRegression(C=32, max_iter=1000)


In [29]:
# The classifier has now been trained. Let's predict the labels for the eval_set texts and see how accurate the classifier is
# We first convert the evaluation texts into embeddings with our sentence_transformer and labels to binary {0,1} labels

# filtering just the text from the pandas dataframe
eval_texts = list(eval_data["content"])
eval_embeddings = sent_encoder.encode(eval_texts)#TODO: get the embeddings of the evaluation reviews (eval_texts) with the sent_encoder 

# converting labels from "POS" and "NEG" into numeric labels (0 and 1) for classification
eval_labels = eval_data["label"].tolist()
eval_labels = [(1 if tl == "POS" else 0) for tl in eval_labels]

In [30]:
# now we call the classifier to predict and score (function "score") the predictions on the evaluation dataset
accuracy = classifier.score(eval_embeddings, eval_labels)
print("Classification accuracy: " + str(accuracy * 100) + "%")


Classification accuracy: 88.0%


In [31]:
# Let's see some predictions on individual texts

# we are generating predictions as probability distributions over the two classes, for each text
predict_probs = classifier.predict_proba(eval_embeddings)

# for a text at "index", we'll display the text itself and the class probability distribution produced by the LR classifier
# play by changing the "index"
index = 100
print(eval_texts[index])
print(predict_probs[index])



 I won't recommend to any body. You have the same way of charging thru your computer. I think is waste
[0.94410279 0.05589721]


...and now, the **language transfer** (zero-shot)

**Zero-shot** means that we are able to make predictions for texts in the target language without seeing any training instances in that language (i.e., our classifier was trained on English reviews). However, because the input features for the classifier are generated by a multilingual text encoder, we can make predictions for any language that our Sentence Transformer (sent_encoder) can encode! (and that's around 100 languages).


In [36]:
# let's create a few reviews in other languages:

# "This is a bad speaker, you can barely hear anything"
rev_de = "Das ist ein ziemlich schlechter Lautsprecher, man kann kaum etwas hören."

# "The keyboard is light, easy to use and actually pretty beautiful. Totally worth the money!"
rev_it = "La tastiera è leggera, facile da usare e in realtà piuttosto bella. Vale assolutamente i soldi!" 

# This USB adapter is so average. Not bad, but I wouldn't give again that money for it. 
rev_hr = "Ovaj USB adapter je onako, prosječan. Nije loš, ali ne bih ponovno dao te novce za njega."

# This is by far the best USB charger I have ever had. My phone is charged in less than an hour, so amazing!
rev_zh = "这是迄今为止我所拥有的最好的USB充电器。我的手机在不到一小时内就充好电了，太神奇了!"
rev_zh2 = "那是拖鞋，但是他說他建議皮鞋"
revs = [rev_de, rev_it, rev_hr, rev_zh2]

In [37]:
# let's encode the reviews with the sentence_transformer 
rev_embeddings = sent_encoder.encode(revs)#TODO: embed the reviews above with the sent_encoder

# now let's classify them and make predictions with our LR classifier
predict_probs = classifier.predict_proba(rev_embeddings)

# let's print each review and the predicted sentiment probabilities
for i in range(len(revs)):
    print(revs[i])
    print(predict_probs[i])
    pred_label = "NEG" if predict_probs[i][0] > predict_probs[i][1] else "POS"  
    print("Predicted label: " + pred_label)
    print()
    

Das ist ein ziemlich schlechter Lautsprecher, man kann kaum etwas hören.
[0.9900897 0.0099103]
Predicted label: NEG

La tastiera è leggera, facile da usare e in realtà piuttosto bella. Vale assolutamente i soldi!
[0.00184052 0.99815948]
Predicted label: POS

Ovaj USB adapter je onako, prosječan. Nije loš, ali ne bih ponovno dao te novce za njega.
[0.62147646 0.37852354]
Predicted label: NEG

那是拖鞋，但是他說他建議皮鞋
[0.90575011 0.09424989]
Predicted label: NEG



In [34]:
# TODO: discuss the expected transfer performance for the above target languages. What could be the main factors
# that determine the success of transfer of a classifier based on the pretrained multilingual encoder from the source language
# (i.e., English) to a concrete target language?

In [35]:
# Answer: 
# I think the main factor of the NEG or POS, is according to the embedding vectorized floats, according to the weight value,
# we can know the relationship between the contexts. And then according to the contexts values, matching the pretrained POS or NEG
# sentences' contexts.